#### Uses external datasets to test the dataset:
- https://www.kaggle.com/azaemon/preprocessed-ct-scans-for-covid19
- https://www.kaggle.com/paultimothymooney/chest-xray-pneumonia

In [1]:
import numpy as np
from pathlib import Path
from keras.models import load_model
from tensorflow.keras.preprocessing import image
from KyleNet import KyleNet

c:\users\kyle\appdata\local\programs\python\python38\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\users\kyle\appdata\local\programs\python\python38\lib\site-packages\numpy\.libs\libopenblas.QVLO2T66WEPI7JZ63PS3HMOHFEY472BC.gfortran-win_amd64.dll
c:\users\kyle\appdata\local\programs\python\python38\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


##### Get the best model from the experiments and COVID/NON-COVID test images

In [2]:
model_path = Path(".").resolve().parent.joinpath("models/Experiment 4.h5")

# Dataset directory
DATASETS = Path("F:").resolve().joinpath("Datasets")

# CT Scans for COVID-19 Classification Dataset
CT_DATASET = DATASETS.joinpath("CT Scans for COVID-19 Classification")
ORIGINAL_COVID_CT = CT_DATASET.joinpath("Original/pCT")
PREPROCESSED_COVID_CT = CT_DATASET.joinpath("Preprocessed/pCT")
ORIGINAL_NONCOVID_CT = CT_DATASET.joinpath("Original/nCT")
PREPROCESSED_NONCOVID_CT = CT_DATASET.joinpath("Preprocessed/nCT")

# Chest X-ray Pneumonia Dataset
CXR_PNEUMONIA_DATASET = Path("F:").resolve().joinpath("Datasets/Chest X-ray Pneumonia/")
CXR_NORMAL = CXR_PNEUMONIA_DATASET.joinpath("test/NORMAL")
CXR_PNEUMONIA = CXR_PNEUMONIA_DATASET.joinpath("test/PNEUMONIA")

##### Load the model

In [3]:
model = load_model(model_path)

# Classes For CNN Model Predictions
classes = {0: "COVID-19", 1: "NON-COVID"}

##### Load KyleNet and change the model to the best experiment model to confirm the accuracy by evaluating it on the testing set

In [40]:
META = Path(".").resolve().parent.joinpath("metadata.csv")
exp4 = KyleNet(META, "", balance_dataset=True)
exp4.model = model
exp4.evaluate()

Found 57830 validated image filenames belonging to 2 classes.
Found 19276 validated image filenames belonging to 2 classes.
151/151 [==============================] - 80s 531ms/step - loss: 0.0014 - accuracy: 0.9999

##### Prediction Function

In [4]:
def GetPrediction(img):
    img = image.load_img(img, target_size=(224, 224))
    img = image.img_to_array(img) / 255
    x = np.expand_dims(img, axis=0)
    prediction = (model.predict(x, batch_size=128) > 0.5).astype("int")
    return classes.get(prediction[0][0])

In [5]:
def PredictDirectory(directory, finding):
    assert finding == "COVID-19" or finding == "NON-COVID", f"Finding should be COVID-19 or NON-COVID, not {finding}"

    if(finding == "COVID-19"):
        inverse_finding = "NON-COVID"
    else:
        inverse_finding = "COVID-19"

    results = []
    for img in directory.glob("*"):
        if img.is_file():
            results.append(GetPrediction(img))

    print(f"Number of {finding} images: {len(results)}\n"
          f"Correctly classified as {finding}: {results.count(finding)}\n"
          f"Misclassified as {inverse_finding}: {results.count(inverse_finding)}")

### CT scan testing

##### COVID-19 Original CT Scans

In [43]:
PredictDirectory(ORIGINAL_COVID_CT, "COVID-19")

Number of COVID-19 images: 4001
Correctly classified as COVID-19: 3508
Misclassified as NON-COVID: 493


##### COVID-19 Pre-processed CT Scans (removed noise, black background)

In [44]:
PredictDirectory(PREPROCESSED_COVID_CT, "COVID-19")

Number of COVID-19 images: 4001
Correctly classified as COVID-19: 1003
Misclassified as NON-COVID: 2998


##### NON-COVID Original CT Scans

In [45]:
PredictDirectory(ORIGINAL_NONCOVID_CT, "NON-COVID")

Number of NON-COVID images: 9979
Correctly classified as NON-COVID: 6479
Misclassified as COVID-19: 3500


##### NON-COVID Pre-processed CT Scans (removed noise, black background)

In [10]:
PredictDirectory(PREPROCESSED_NONCOVID_CT, "NON-COVID")

Number of NON-COVID images: 9979
Correctly classified as NON-COVID: 9484
Misclassified as COVID-19: 495


### CXR image testing

##### NORMAL CXR images

In [7]:
PredictDirectory(CXR_NORMAL, "NON-COVID")

Number of NON-COVID images: 234
Correctly classified as NON-COVID: 184
Misclassified as COVID-19: 50


##### PNEUMONIA CXR images

In [8]:
PredictDirectory(CXR_PNEUMONIA, "NON-COVID")

Number of NON-COVID images: 390
Correctly classified as NON-COVID: 348
Misclassified as COVID-19: 42
